# Text Cleaning and Vectorization

This notebook prepares the `reviews.text` field for NLP modeling. I clean the text, analyze review length, and vectorize it using CountVectorizer and TF-IDF. Each artifact is saved separately for downstream ML, DL, and fine-tuning workflows.

In [ ]:
# Text Cleaning and Vectorization

import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [ ]:
# Downloads
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Setup
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
df = pd.read_parquet("/content/cleaned_reviews_with_categories.parquet")
print(" Loaded:", df.shape)

 Loaded: (9480, 30)


In [ ]:
# Text cleaning function
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    # Remove digits
    text = re.sub(r"\d+", '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Normalize whitespace
    text = re.sub(r"\s+", ' ', text).strip()
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    # Lemmatize
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

# Apply cleaning
df['clean_text'] = df['reviews.text'].apply(clean_text)

In [ ]:
# Review length analysis
df['text_length'] = df['clean_text'].str.split().apply(len)
df['is_short'] = df['text_length'] < 5
print(f"Short reviews (<5 words): {df['is_short'].sum()}")

Short reviews (<5 words): 395


In [ ]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import joblib

count_vec = CountVectorizer(max_features=5000)
X_count = count_vec.fit_transform(df['clean_text'])
joblib.dump(X_count, "/content/X_count_vectorized.pkl")
joblib.dump(count_vec, "/content/count_vectorizer.pkl")
print("Saved CountVectorizer and matrix.")

Saved CountVectorizer and matrix.


In [ ]:
# TF-IDF Vectorizer
tfidf_vec = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vec.fit_transform(df['clean_text'])
joblib.dump(X_tfidf, "/content/X_tfidf_vectorized.pkl")
joblib.dump(tfidf_vec, "/content/tfidf_vectorizer.pkl")
print("Saved TF-IDF and matrix.")

Saved TF-IDF and matrix.


I cleaned and vectorized the review text. Each artifact was saved separately:

- `cleaned_reviews_with_text.parquet`: full dataset with cleaned text
- `X_count_vectorized.pkl`: CountVectorizer matrix
- `count_vectorizer.pkl`: fitted CountVectorizer
- `X_tfidf_vectorized.pkl`: TF-IDF matrix
- `tfidf_vectorizer.pkl`: fitted TF-IDF

These files are ready for ML, DL, and fine-tuning workflows.